# Import packages
## this notebook uses some functions that we have not encountered yet
* '_sounddevice_' is a package that lets you access your computer's microphone and speaker.
    > Unfortunately this only works when python is running locally on your computer. <br>
    So we have commented that line of code out (the package does not exist in collaboratory but will be useful when we load python locally on your computer) <br>
    Google collaboratory notebooks cannot access your computer's local environment to communicate with your microphone, etc. Instead of using this package to perform the experiment, for now we will look at the results of data that I collected by doing a simple psychophysical experiment on myself.
* '_matplotlib.pyplot_' and '_seaborn_' are both packages that contain lots of functions for plotting _arrays_ of numbers (such as a waveform recorded via the microphone)
    > https://seaborn.pydata.org/
* '_scipy_' is a 'scientific' package that contains functions for processing arrays of numbers (like filtering them or detecting peaks in a signal). Here we will use it to get a _spectrogram_ of our waveform.
    > https://en.wikipedia.org/wiki/Spectrogram#:~:text=A%20spectrogram%20is%20a%20visual,sonographs%2C%20voiceprints%2C%20or%20voicegrams.
* '_pathlib_' is a package that is used to work with file/directory paths to pass files into and out of functions (such as saving an array to a waveform file on your computer or loading an array from a file)
* '_google.colab_' is a package that contains a module (called _drive_) that enables you to interact with the files in your google drive
* '_IPython.display_' is a package that contains a module ('Audio') to play arrays in the notebook workspace as audio files out of your microphone


In [ ]:
import numpy as np
# import sounddevice as sd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from pathlib import Path
from google.colab import drive
from IPython.display import Audio


# To load the data we will need to connect to your google drive. 
* Remember to make sure that the data from the experiment is uploaded to your Google drive into a folder named '__data_VocalTurnTaking__'. 

To mount Google Drive, run the below code and go to the link to retireve the authorization code. Paste the authorization code in the box created by running the code and press enter.


In [ ]:
drive.mount('/content/gdrive')



Once mounted successfully, your entire Google Drive files should be accessible under /content/gdrive/My\ Drive/
<br>
> After mounting Google Drive, you can download datasets into Google Drive to use in the Colab session, or save outputs of the session into your Google Drive.

> You can also view the contents of your drive directory by expanding the file folder icon to the left.


In [ ]:
!ls /content/gdrive/My\ Drive


Next we will navigate to 'My Drive' using the 'change directory' command (```%cd```)

In [ ]:
%cd gdrive/My\ Drive

We will now create a 'pathlib' object that essentially saves the google drive directory as a string that we can pass to 'load' functions.

In [ ]:
savepath = Path('data_VocalTurnTaking')

# Now that we have access to the data for the experiment, let's explore the experiment and its data.

## Run the following cell to specify some "_metadata_" about the data collected.
> https://en.wikipedia.org/wiki/Metadata

In [ ]:
sampleRate = 20000
duration = 3

## Experiment Part I) First, a human 'call' was recorded and played to a human test subject.  

In [ ]:
# load the call signal/waveform and store it as a local variable named 'call'
call = np.load(savepath / 'Hello.npy').flatten()

# 'flatten()' changes the shape of the array... 
# not an important detail for now, but something to keep in mind for future debugging

We can play the array using the 'Audio' module. When you run the cell below it crates a graphical interface below the code cell with a play button, a volume slider, and the option to download (unecessary since we already have this audio file stored). 

In [ ]:
# Generate a player for mono sound
Audio(call,rate=sampleRate)

### Waveforms/signals can be visualized in several different ways. Let's examine two ways to visualize the call that you just heard:
> (1) as a raw waveform; the units are usually 'Volts.'<br>
> (2) as a '_spectrogram_'; the units are frequency ('Hertz').<br>

Both of these are plotted as a function of time.

The code below creates a time vector to plot the raw waveform against. It contains the same number of points as the 'call' waveform. It starts at time = 0 seconds and ends at time = 3 seconds. It is convension to name this 'xtime' since it will be plotted on the x-axis.

In [ ]:
xtime = np.linspace(0, duration, sampleRate * duration) 

We can now use the matplotlib.pyplot function 'plot' to plot the amplitude of the call signal as a function of time.

In [ ]:
# first, make a figure
hfig = plt.figure()

# The following function takes 'x' and 'y' and plots them:
plt.plot(xtime, call) 
# you can provide other elements to the function
# by adding them inside the parentheses (separated by a comma)

# BELOW is where you will insert functions to add labels to the plot



You can change some of the aesthetic features of the plot by passing arguments to the plot() function. 
> change the color of the line to green by adding ``` color = 'green' ``` to the list of elements passed to the plot function. Try other colors as well. 

You can also add labels to the plot
> Add a line of code to the cell above that uses the ``` plt.xlabel() ``` function to add a label for the x axis describing what is being plotted (time in seconds). <br>
>> use the following syntax: ``` plt.xlabel('time, seconds') ``` <br>

> Do the same for the y label using ``` plt.ylabel() ``` to specify that the amplitude is in Volts. 

Another helpful way to visualize audio signals is to plot a spectrogram. The '_signal_' module that we imported at the beginning of the notebook has a function for calculating the spectrogram of the signal.
> Where specified in the code cell below, add axes labels like you did for the previous plot. (The Y label will be different since this is a plot of frequency against time rather than amplitdue against time).

In [ ]:
# First we need to specify parameters to input into the spectrogram function:
windur = 0.01 # the duration of each chunk (window) to calculate the power across frequencies
winsamp = int(windur*sampleRate) # transforms the window duration into samples

# next, we use the spectrogram() function to calculate the spectrogram
f, t, Sxx = signal.spectrogram(call,
                               fs = sampleRate,
                               window = 'hamming',
                               nperseg = winsamp,
                               noverlap = winsamp/2
                              )
# f is an array of all the frequency bins analyzed
# t is an array of all the time windows analyzed
# Sxx is a matrix of the power at a given frequency in a given time window

# the 'plt' module has a function that plots this kind of 3-dimensional data
plt.pcolormesh(t, f, np.log(Sxx), cmap='plasma') 

# ADD LABELS FOR THE X AND Y AXES HERE:
# remember, a spectrogram is a plot of frequency across time




## Experiment Part II) Second, the call was played 20 times (20 trials). On each trial, a human test subject responded to the call with a response. The vocalization of the test subject was recorded on each trial and saved as an array. 

In [ ]:
# load the matrix of response signals/waveforms and store it as a local variable named 'response_single'
ResponseSingle = np.load(savepath / 'ResponseSingle.npy')

Let's look at the shape of this matrix by using the ``` np.shape() ``` function below.

In [ ]:
np.shape(ResponseSingle)

This means that the matrix has 20 rows and 60000 columns. Each row is the response on a single trial. Each column is a sample of the amplitude of the signal at each moment in time throughout each trial. 
> The number of samples is equal to the duration of the recording on each trial (3 seconds) multiplied by the sampling rate (20000 Hertz). This metadata was stored as variables earlier and can be used to calculate the expected number of samples in each row of the matrix:

In [ ]:
SamplesPerRow = duration * sampleRate
print('Each row should have this many samples: ')
print(SamplesPerRow)

We can look at just one row at a time by assigning a single row to a new variable. <br>
The cell below assigns the first row (number '0') to a variable called 'trial0'.

In [ ]:
trial0 = ResponseSingle[0,:]

Using the examples from Part I of the experiment above, create additional code cells below to write code that makes the following plots of the data from the first trial that is saved as the variable '_trial0_':
* 1) Plot the amplitude against time
* 2) Plot the frequency against time for the spectrogram of the signal.

You can even copy and paste most of the code written in Part I into new code cells below and just change the variable for the signal being plotted. The time variable (xtime) that we created before should be the same here.

### Analyzing data from Part II:
> What is some of the important information that we want to find out?
>> One of the main pieces of information that we want to calculate from the response data is the distribution of repsonse latencies (ie. the time between the start of the call and the start of the response).

To calculate this information, we need to first know the onset of the call (in seconds). You can get an approximate estimate of this from looking at the plots you made in Part I.
> Look at the plots you made to write an approximate time of call onset in the Markdown cell below:

To calculate a more exact estimate of the call onset, we will write code that finds when the signal power across frequencies (the absolute magnitude) increases above some threshold value.
> To do this, we will use the spectrogram function to get the power at each frequency at each time. Then, for each moment in time, we will use the ``` np.sum() ``` function to add the power across all frequencies. We will call this the __call_envelope__

In [ ]:
f, t, Sxx = signal.spectrogram(call,
                               fs = sampleRate,
                               window = 'hamming',
                               nperseg = winsamp,
                               noverlap = winsamp/2
                              )

call_envelope = np.sum(Sxx,0) 
# the '0' element passed to this function specifies to sum across the first element of the Sxx matrix

Now we can plot the call_envelope across time to see what it looks like. Remember that '_t_' (one of the outputs of the spectrogram() function) is a vector of the time values for each sample in call_envelope.

In [ ]:
# make the figure
hfig = plt.figure()

# plot the call_envelope against time and color it whatever color you want
plt.plot(t, call_envelope, color = 'black')

# pick a threshold value and 
# plot a horizontal line at that value 
# to see where the call_envelope first goes above that value.
threshold = 0.0001
plt.hlines(threshold, 0, 3, color = 'red', linestyle = '--')

We can ask the computer to calculate when the call_envelope rises above the threshold value. We will call this the 'onset' of the call.
> To do this we use the '_np.min()_' and '_np.max()_' functions.

> We will also need to use a comparison operator ``` > ``` (_greater than_) to find where the call_envelope is greater than the threshold value.

In [ ]:
SortingArray = call_envelope > threshold

Print the _SortingArray_ using the code cell below to see what the information contained in this '_boolean_' array looks like. 
> a 'boolean' is an array of 'True' and 'False'

Next, we will filter the time vector '_t_' using this SortingArray and use the np.min() function to find the earliest time at which the call_evelope is above the threshold value.

In [ ]:
TimesAboveThreshold = t[SortingArray]
call_onset = np.min(TimesAboveThreshold)

In [ ]:
print('the call onset is at ')
print(call_onset)
print('seconds')

### Now, that we know the call onset, let's use the same methods to calculate the onset of each response from the test subject
> To do this we will use a for loop to perform the calculation on each trial of the response matrix.

In [ ]:
# first, we will create an empty array to put the data 
# about the envelope of the response on each trial.
response_envelope_all = []

# next we will loop through the response matrix 
# and calculate the envelope of the response on each trial
# and put that envelope in the matrix using the .append() function.
for response in ResponseSingle:
    f, t, Sxx = signal.spectrogram(response,
                               fs = sampleRate,
                               window = 'hamming',
                               nperseg = winsamp,
                               noverlap = winsamp/2
                              )
    response_envelope_this_trial = np.sum(Sxx,0)
    response_envelope_all.append(response_envelope_this_trial)
response_envelope_all = np.asarray(response_envelope_all).T


Now we can plot the response_envelope_all matrix against time to visualize the data. Include a line that shows the threhold value. Notice that without any 'color' argument passed to the plot function, each trial (row of the matrix) is automatically plotted in a different color. 

In [ ]:
hfig = plt.figure()
plt.plot(t,response_envelope_all);
plt.ylabel('power')
plt.xlabel('seconds')

We will now use a for loop to calculate the response onset for each trial using the data in the response_envelope_all matrix that we just created.

In [ ]:
response_onset = []
for response in response_envelope_all.T:
  SortingArray = response > threshold
  onset = np.min(t[SortingArray])
  response_onset.append(onset)
response_onset = np.asarray(response_onset).T


 Now we have an array of the response time (in seconds) on each trial, which you can see by printing the array below.

In [ ]:
print(response_onset)

How would you calculate the response latency on each trial? 
> First, use the Markdown cell below to write in words the calculation that needs to be done.

> Second, use the Code cell below that to write code that performs the calculation and displays the result.

We can plot the call_envelope and the response_envelope overlaid on the same plot in order to visualize and make an estimate of the approximate response latency to the call that should have been calculated above.
* call_envelope is plotted in red and response_envelope_all trials are plotted in black

In [ ]:
plt.figure()
plt.plot(t,call_envelope,color = 'red')
plt.plot(t,response_envelope_all,color = 'black');
plt.xlabel('seconds')
plt.ylabel('power')

# To be continued....